#Generator Input

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import joblib
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

class AutoEncoder(nn.Module):
    def __init__(self, input_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 8192),
            nn.ReLU(),
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.Linear(4096, 8192),
            nn.ReLU(),
            nn.Linear(8192, input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return latent, reconstructed

class CustomDataset(Dataset):
    def __init__(self, mel_dir, mfcc_dir):
        self.mel_files = sorted([os.path.join(mel_dir, f) for f in os.listdir(mel_dir) if f.endswith(".npy")])
        self.mfcc_files = sorted([os.path.join(mfcc_dir, f) for f in os.listdir(mfcc_dir) if f.endswith(".npy")])
        if not self.mel_files or not self.mfcc_files:
            raise ValueError("No .npy files found!")

    def __len__(self):
        return min(len(self.mel_files), len(self.mfcc_files))

    def __getitem__(self, idx):
        mel = torch.tensor(np.load(self.mel_files[idx]), dtype=torch.float32)
        mfcc = torch.tensor(np.load(self.mfcc_files[idx]), dtype=torch.float32)
        return mel, mfcc, os.path.basename(self.mel_files[idx])

# ✅ Paths
AUTOENCODER_MODEL_PATH = r"C:\Users\cl502_11\MG\Models\VQ-VAE + GANs\all hp tuned.joblib"
INPUT_MEL_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\Latents for 2nd Model\INPUT\mel_spectrograms"
INPUT_MFCC_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\Latents for 2nd Model\INPUT\mfccs"
MY_LATENT_SAVE_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\GANs Inputs\Generator"
os.makedirs(MY_LATENT_SAVE_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Fix your trained input dimension paaji!
trained_input_dim = 30456  # 🚀 This MUST match your model training input size

model = AutoEncoder(trained_input_dim).to(device)
state_dict = joblib.load(AUTOENCODER_MODEL_PATH)
model.load_state_dict(state_dict)
model.eval()
for param in model.parameters():
    param.requires_grad = False
print(f"✅ Model loaded with input_dim={trained_input_dim} and frozen!\n")

def extract_latents(dataset, save_dir):
    for mel, mfcc, filename in tqdm(dataset, desc="Extracting Latents"):
        mel = mel.to(device)
        mfcc = mfcc.to(device)

        # 🔥 Concatenate along feature axis
        combined = torch.cat((mel, mfcc), dim=0)  # Shape (mel_channels + mfcc_channels, time_steps)

        # 🔥 Now pad this combined to match the width (time_steps) that gives 30456 after flatten
        # Example: If combined is (141, 216), total = 141 * 216 = 30456 ✅
        expected_channels = 141
        expected_timesteps = 216
        current_channels, current_timesteps = combined.shape

        # Pad channels or time-steps if necessary (just to be safe)
        if current_channels != expected_channels or current_timesteps != expected_timesteps:
            padded = F.pad(combined, (0, expected_timesteps - current_timesteps, 0, expected_channels - current_channels))
        else:
            padded = combined

        # 🔥 Flatten correctly
        combined_flat = padded.flatten().unsqueeze(0).to(device)  # Shape [1, 30456]

        with torch.no_grad():
            latent, _ = model(combined_flat)

        latent_np = latent.squeeze(0).cpu().numpy()
        latent_filename = filename.replace('.npy', '_latent.npy')
        np.save(os.path.join(save_dir, latent_filename), latent_np)

    print(f"\n✅ Latents saved at {save_dir}")

# ✅ Extraction Call
my_dataset = CustomDataset(INPUT_MEL_DIR, INPUT_MFCC_DIR)
extract_latents(my_dataset, MY_LATENT_SAVE_DIR)


✅ Model loaded with input_dim=30456 and frozen!



Extracting Latents: 100%|██████████| 1/1 [00:00<00:00,  3.57it/s]


✅ Latents saved at C:\Users\cl502_11\MG\Feature Extraction\GANs Inputs\Generator


#Discriminator Input

In [ ]:
VQVAE_MODEL_PATH = r"C:\Users\cl502_11\MG\Models\VQ-VAE + GANs\all hp tuned.joblib"  # Replace with your trained VQ-VAE model
KK_MEL_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\DataChunk1 (29 Files)\80_10_10\mel_spectrograms\train"
KK_MFCC_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\DataChunk1 (29 Files)\80_10_10\mfccs\train"
REAL_LATENT_SAVE_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\GANs Inputs\Discriminator"

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import joblib
from torch.utils.data import Dataset
from tqdm import tqdm

# ✅ Full VQ-VAE Model Definition (Matches saved model)
class VQVAE(nn.Module):
    def __init__(self, input_dim):
        super(VQVAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 8192),
            nn.ReLU(),
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024)
        )
        self.decoder = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.Linear(4096, 8192),
            nn.ReLU(),
            nn.Linear(8192, input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return latent, reconstructed

# ✅ Dataset for KK's Mel + MFCC
class KKDataset(Dataset):
    def __init__(self, mel_dir, mfcc_dir):
        self.mel_files = sorted([os.path.join(mel_dir, f) for f in os.listdir(mel_dir) if f.endswith(".npy")])
        self.mfcc_files = sorted([os.path.join(mfcc_dir, f) for f in os.listdir(mfcc_dir) if f.endswith(".npy")])
        if not self.mel_files or not self.mfcc_files:
            raise ValueError("No .npy files found!")

    def __len__(self):
        return min(len(self.mel_files), len(self.mfcc_files))

    def __getitem__(self, idx):
        mel = torch.tensor(np.load(self.mel_files[idx]), dtype=torch.float32)
        mfcc = torch.tensor(np.load(self.mfcc_files[idx]), dtype=torch.float32)
        return mel, mfcc, os.path.basename(self.mel_files[idx])

# ✅ Paths
VQVAE_MODEL_PATH = r"C:\Users\cl502_11\MG\Models\VQ-VAE + GANs\all hp tuned.joblib"  # Replace with your trained VQ-VAE model
KK_MEL_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\DataChunk1 (29 Files)\80_10_10\mel_spectrograms\train"
KK_MFCC_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\DataChunk1 (29 Files)\80_10_10\mfccs\train"
REAL_LATENT_SAVE_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\GANs Inputs\Discriminator"
os.makedirs(REAL_LATENT_SAVE_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_input_dim = 30456  # Match your training

# ✅ Load the Full VQ-VAE Model
vqvae = VQVAE(trained_input_dim).to(device)
vqvae_state = joblib.load(VQVAE_MODEL_PATH)
vqvae.load_state_dict(vqvae_state)
vqvae.eval()
print(f"✅ Full VQ-VAE Model Loaded and Frozen!\n")

for param in vqvae.parameters():
    param.requires_grad = False

# ✅ Latent Extraction Function
def extract_kk_latents(dataset, save_dir):
    for mel, mfcc, filename in tqdm(dataset, desc="Extracting KK Real Latents"):
        mel = mel.to(device)
        mfcc = mfcc.to(device)

        combined = torch.cat((mel, mfcc), dim=0)
        expected_channels = 141
        expected_timesteps = 216
        cur_channels, cur_timesteps = combined.shape

        if cur_channels != expected_channels or cur_timesteps != expected_timesteps:
            padded = F.pad(combined, (0, expected_timesteps - cur_timesteps,
                                      0, expected_channels - cur_channels))
        else:
            padded = combined

        combined_flat = padded.flatten().unsqueeze(0).to(device)

        with torch.no_grad():
            latent, _ = vqvae(combined_flat)  # ✅ Extract through full model to access encoder

        latent_np = latent.squeeze(0).cpu().numpy()
        latent_filename = filename.replace('.npy', '_kk_latent.npy')
        np.save(os.path.join(save_dir, latent_filename), latent_np)

    print(f"\n✅ KK Real Latents saved at {save_dir}")

# ✅ Run Extraction
kk_dataset = KKDataset(KK_MEL_DIR, KK_MFCC_DIR)
extract_kk_latents(kk_dataset, REAL_LATENT_SAVE_DIR)


✅ Full VQ-VAE Model Loaded and Frozen!



Extracting KK Real Latents: 100%|██████████| 767/767 [00:10<00:00, 70.77it/s]


✅ KK Real Latents saved at C:\Users\cl502_11\MG\Feature Extraction\GANs Inputs\Discriminator


#FINAL GANs model

In [ ]:
# ✅ Best hyperparameters from Optuna (Paste these manually)
best_params = {
    'hidden_dim': 1476,
    'dropout': 0.49094941961091343,
    'lr_gen': 0.00024773548667528797,
    'lr_disc': 3.314136329501845e-05,
    'batch_size': 32
}


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import joblib
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GEN_INPUT_PATH = r"C:\Users\cl502_11\MG\Feature Extraction\GANs Inputs\Generator\MW_latent.npy"
DISC_REAL_DIR = r"C:\Users\cl502_11\MG\Feature Extraction\GANs Inputs\Discriminator"
SAVE_DIR = r"C:\Users\cl502_11\MG\Models\GANs\Final_Training"
os.makedirs(SAVE_DIR, exist_ok=True)

latent_dim = 1024
EPOCHS = 50  # 🔥 Set your desired epochs

# ✅ Dataset for Discriminator Real Samples
class RealLatentDataset(Dataset):
    def __init__(self, directory):
        self.files = sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.npy')])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        return torch.tensor(np.load(self.files[idx]), dtype=torch.float32)

# ✅ Generator and Discriminator Classes
class Generator(nn.Module):
    def __init__(self, latent_dim, hidden_dim, dropout):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, latent_dim)
        )
    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, latent_dim, hidden_dim, dropout):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

# ✅ Load Best Hyperparameters
hidden_dim = best_params['hidden_dim']
dropout = best_params['dropout']
lr_gen = best_params['lr_gen']
lr_disc = best_params['lr_disc']
batch_size = best_params['batch_size']

# ✅ Model Initialization
G = Generator(latent_dim, hidden_dim, dropout).to(device)
D = Discriminator(latent_dim, hidden_dim, dropout).to(device)

optimizer_G = optim.Adam(G.parameters(), lr=lr_gen)
optimizer_D = optim.Adam(D.parameters(), lr=lr_disc)
criterion_adv = nn.BCELoss()
criterion_recon = nn.L1Loss()

# ✅ Load Generator Input (Your Latent)
gen_input = torch.tensor(np.load(GEN_INPUT_PATH), dtype=torch.float32).unsqueeze(0).to(device)

# ✅ Load Discriminator Real Data
real_dataset = RealLatentDataset(DISC_REAL_DIR)
real_loader = DataLoader(real_dataset, batch_size=batch_size, shuffle=True)

# ✅ Training Loop
for epoch in range(1, EPOCHS + 1):
    G.train()
    D.train()
    epoch_loss = 0

    for real_data in tqdm(real_loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        real_data = real_data.to(device)

        # 🔥 Train Discriminator
        optimizer_D.zero_grad()
        real_preds = D(real_data)
        fake_data = G(gen_input).detach().repeat(real_data.size(0), 1)
        fake_preds = D(fake_data)

        d_real_loss = criterion_adv(real_preds, torch.ones_like(real_preds))
        d_fake_loss = criterion_adv(fake_preds, torch.zeros_like(fake_preds))
        d_loss = (d_real_loss + d_fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # 🔥 Train Generator
        optimizer_G.zero_grad()
        gen_output = G(gen_input)
        adv_loss = criterion_adv(D(gen_output), torch.ones((1, 1), device=device))
        recon_loss = criterion_recon(gen_output, real_data[0].unsqueeze(0))  # Optional recon loss
        g_loss = adv_loss + 0.5 * recon_loss
        g_loss.backward()
        optimizer_G.step()

        epoch_loss += g_loss.item()

    print(f"✅ Epoch {epoch} Completed - Generator Loss: {epoch_loss:.4f}")

# ✅ Save Final Models
joblib.dump(G.state_dict(), os.path.join(SAVE_DIR, "Generator_final.joblib"))
joblib.dump(D.state_dict(), os.path.join(SAVE_DIR, "Discriminator_final.joblib"))
print("\n✅ Models Saved Successfully at", SAVE_DIR)


Epoch 1/50: 100%|██████████| 24/24 [00:00<00:00, 27.28it/s]


✅ Epoch 1 Completed - Generator Loss: 10249.6728


Epoch 2/50: 100%|██████████| 24/24 [00:00<00:00, 41.06it/s]


✅ Epoch 2 Completed - Generator Loss: 4991.5491


Epoch 3/50: 100%|██████████| 24/24 [00:00<00:00, 36.98it/s]


...

✅ Epoch 48 Completed - Generator Loss: 3419.8038


Epoch 49/50: 100%|██████████| 24/24 [00:00<00:00, 39.77it/s]


✅ Epoch 49 Completed - Generator Loss: 3017.8166


Epoch 50/50: 100%|██████████| 24/24 [00:00<00:00, 39.34it/s]

✅ Epoch 50 Completed - Generator Loss: 2923.3707

✅ Models Saved Successfully at C:\Users\cl502_11\MG\Models\GANs\Final_Training
